In [42]:
import pandas as pd
import numpy as np
import pyreadstat
import warnings

In [43]:
warnings.filterwarnings("ignore", category=FutureWarning, message=".*concatenation with empty or all-NA entries.*")
file_path = './DATA/data-check.sav'
df, meta = pyreadstat.read_sav(file_path)
current_year = 2024
pd.options.mode.copy_on_write = True

Logic 1 YearsOld

In [44]:
#yearold,age
col_1 = ['SbjNum', 'S2_YEAR', 'S2_TUOI']
df_1 = df[col_1]  

df_1['Error_1'] = "Kiểm tra Year + Age = 2024 của đáp viên"

def Logic_1(row):
    error_message = set() 
    YearOld=row['S2_YEAR']
    Age=row['S2_TUOI']
    Calculated_Age = current_year - YearOld
    if(Calculated_Age!=Age):
        error_message.add("Kiểm tra lại tuổi")
    return pd.Series([error_message])

df_1[['Error_1']]=df_1.apply(Logic_1,axis=1)

Result_1=df_1[(df_1['Error_1'].apply(lambda x: len(x)>0))]
columns=['Error_1','SbjNum'] + [col for col in Result_1.columns if col not in ['Error_1','SbjNum']]
Result_1 = Result_1[columns]
print(Result_1)  

               Error_1       SbjNum  S2_YEAR  S2_TUOI
0  {Kiểm tra lại tuổi}  157630659.0   1999.0     32.0


Logic 2 Nhóm tuổi 

    Dưới 21 tuổi	1
    21 - 28 tuổi	2
    29 - 38 tuổi	3
    39 - 48 tuổi	4
    Trên 48 tuổi	5

In [45]:
# SbjNum, S2_TUOI,S2_NHOMTUOI


col_2=['SbjNum','S2_TUOI','S2_NHOMTUOI']
df_2=df[col_2]

def Logic_2(row):
    error_message = set() 
    Age=row['S2_TUOI']
    AgeGroup=row['S2_NHOMTUOI']

    if((AgeGroup==1 and Age>=21)
       or (AgeGroup==2 and Age not in {21,22,23,24,25,26,27,28})
       or (AgeGroup==3 and Age not in {29,30,31,32,33,34,35,36,37,38})
       or (AgeGroup==4 and Age not in {39,40,41,42,43,44,45,46,47,48})
       or (AgeGroup==5 and Age<48)
       ):
        error_message.add("Kiểm tra lại nhóm tuổi")
    return pd.Series([error_message])


df_2[[ 'Error_2']]=df_2.apply(Logic_2,axis=1)

Result_2=df_2[(df_2['Error_2'].apply(lambda x: len(x)>0))]
columns=['Error_2','SbjNum'] + [col for col in Result_2.columns if col not in ['Error_2','SbjNum']]
Result_2 = Result_2[columns]
print(Result_2)  

                    Error_2       SbjNum  S2_TUOI  S2_NHOMTUOI
0  {Kiểm tra lại nhóm tuổi}  157630659.0     32.0          4.0


Logic 3 Thu nhập

    Trên 20,000,000 VND 	            1	TIẾP TỤC		
    13,500,001 - 20,000,000 VND	    2	||	
    8,500,001 - 13,500,000 VND 	    3	||	
    5,500,001 - 8,500,000 VND 	    4	NGƯNG		
    Dưới 5,500,001 VND	            5	||	
    Từ chối trả lời/ Không biết	    6	||	


In [46]:
# SbjNum, S5_INCOME,S5


col_3=['SbjNum','S5_INCOME','S5']
df_3=df[col_3]

def Logic_3(row):
    error_message = set() 
    InCome=row['S5_INCOME']
    InComeGroup=row['S5']

    if((InComeGroup==1 and InCome<=20)
       or (InComeGroup==2 and (InCome<13.5 or InCome>20) )
       or (InComeGroup==3 and (InCome<8.5 or InCome>13.5))
       ):
        error_message.add("Kiểm tra lại thu nhập")
    

    if (InComeGroup in {4,5,6}):
        error_message.add("Kiểm tra lại nhóm thu nhập")

    return pd.Series([error_message])


df_3[[ 'Error_3']]=df_3.apply(Logic_3,axis=1)

Result_3=df_3[(df_3['Error_3'].apply(lambda x: len(x)>0))]
columns=['Error_3','SbjNum'] + [col for col in Result_3.columns if col not in ['Error_3','SbjNum']]
Result_3 = Result_3[columns]
print(Result_3)  

                        Error_3       SbjNum  S5_INCOME   S5
0  {Kiểm tra lại nhóm thu nhập}  157630659.0       15.0  4.0


Logic 4 :

    NGƯNG NẾU S9 KHÔNG CHỌN CODE 1
    S9_08 chọn nhưng S_S9_8 không có dữ liệu

In [47]:
#SbjNum,S9_O1,S9_O8,S_S9_8
col_4=['SbjNum','S9_O1','S9_O8','S_S9_8']
df_4=df[col_4]

def Logic_4(row):
    error_message = set() 
    S9_1=row['S9_O1']
    S9_8=row['S9_O8']
    S9_8_OE=row['S_S9_8']

    if(S9_1!=1):
        error_message.add("Kiểm tra lại S9 không có code 1 được chọn")

    if(S9_8==1 and pd.isna(S9_8_OE)):
        error_message.add("Kiểm tra lại S9_08 Code 1 data S9_08_OE trống")
    return pd.Series([error_message])

df_4[[ 'Error_4']]=df_4.apply(Logic_4,axis=1)

Result_4=df_4[(df_4['Error_4'].apply(lambda x: len(x)>0))]
columns=['Error_4','SbjNum'] + [col for col in Result_4.columns if col not in ['Error_4','SbjNum']]
Result_4 = Result_4[columns]
print(Result_4)  

Empty DataFrame
Columns: [Error_4, SbjNum, S9_O1, S9_O8, S_S9_8]
Index: []


Export Error file Excel

In [48]:
def convert_to_value_labels(df, meta):
    df_copy = df.copy()
    for col in df_copy.columns:
        if col in meta.variable_value_labels:
            value_labels = meta.variable_value_labels[col]
            try:
                df_copy[col] = df_copy[col].map(value_labels).fillna(df_copy[col])
            except Exception as e:
                print(f"Error processing column {col}: {e}")
    return df_copy


output_file_path = './Excel-output-LogicCheck/Error.xlsx'

# Create DataFrames list with corresponding sheet names
data_frames = {
    f'Result_{i}': globals()[f'Result_{i}'] for i in range(1, 200)
    if f'Result_{i}' in globals() and not globals()[f'Result_{i}'].empty and len(globals()[f'Result_{i}']) > 0
}

# Print out the keys of data_frames for debugging
print(f"DataFrames to be exported: {list(data_frames.keys())}")

# Export to Excel
with pd.ExcelWriter(output_file_path) as writer:
    # Create a dictionary to hold labeled DataFrames
    labeled_data_frames = {}
    
    # Convert and store labeled DataFrames
    for sheet_name, df in data_frames.items():
        if not df.empty:
            labeled_df = convert_to_value_labels(df, meta)
            labeled_data_frames[sheet_name] = labeled_df

    # Create the results list from labeled DataFrames
    results = [(labeled_data_frames[sheet_name], f'Logic {sheet_name.split("_")[1]}') for sheet_name in labeled_data_frames]

    # Create overview sheet data
    overview_data = {
        'Logic Name': [],
        'Row Count': [],
        'Logic set/question (What)': []
    }

    # Populate overview data
    for result, sheet_name in results:
        if not result.empty:
            unique_errors = set()
            for col in result.columns:
                if col.startswith('Error_'):
                    for error_list in result[col].dropna():
                        if isinstance(error_list, list):
                            unique_errors.update(str(e) for e in error_list)
                        elif isinstance(error_list, str):
                            unique_errors.add(error_list)
            overview_data['Logic Name'].append(sheet_name)
            overview_data['Row Count'].append(len(result))
            overview_data['Logic set/question (What)'].append(", ".join(unique_errors))

    if not overview_data['Logic Name']:
        overview_data['Logic Name'].append('No data available')
        overview_data['Row Count'].append(0)
        overview_data['Logic set/question (What)'].append('No data available')

    overview_df = pd.DataFrame(overview_data)
    overview_df.to_excel(writer, sheet_name='Overview', index=False)

    # Write the non-empty labeled results to the Excel file
    for result, sheet_name in results:
        if not result.empty:
            result.to_excel(writer, sheet_name=f'{sheet_name}', index=False)

DataFrames to be exported: ['Result_1', 'Result_2', 'Result_3']
